In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# load weights into new model
model  = load_model("MNIST.h5")

In [13]:
import numpy as np
import cv2

#change to your video path
cap = cv2.VideoCapture(0)

while True:
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    x, y, w, h = 0, 0, 300, 300
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh1 = thresh1[y:y + h, x:x + w]
    
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        
        if cv2.contourArea(contour) > 2500:
            
            x, y, w, h = cv2.boundingRect(contour)
            
            newImage = thresh1[y:y + h, x:x + w]
            newImage = cv2.resize(newImage, (28, 28))
            
            # Reshape
            newImage = newImage.reshape(1,28, 28, 1)
            
            # prepare pixel data
            newImage = newImage.astype('float32')
            newImage = newImage / 255.0
            
            predict_value = model.predict(newImage)[0]
               
            pred =  np.argmax(predict_value)

    x, y, w, h = 0, 0, 300, 300
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.putText(frame, "Prediction : " + str(pred), (10, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Display the resulting frame
    cv2.imshow("Originl",frame)
    
    cv2.imshow("threshold",thresh1)
    
    #cv2.imshow('Croped frame',imgCropped)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()